# More about optimization

In previous lessons, we use *optimization*, *learning*, and *training* interchangeably. They all mean finding the best set of model parameters. However, optimization has a broader meaning -- any applications in which we want to find parameters that best meet our goals belong to optimization. It is itself a topic of study and research.

Gradient descent is an entry-level optimization method. The first part of this lesson will introduce stochastic gradient descent, an optimization method derived from vanilla gradient descent. In the second half of this lesson, we will talk about using the optimizers from SciPy. SciPy is a well-known Python package for scientific
computing. It has several advanced optimizers that may be too complicated for us to write from scratch. Using third-party packages like SciPy can save us a lot of time.

Our goal is to give you a taste of different optimizations and how to use third-party packages. We will refrain from digging too deep into the math.

##### Warm-up exercise

Consider the following problem: *a to-be-built rectangular fence will enclose $120$ square feet. The material cost for the long and short sides will be $\$5$ and $\$6$ per foot, respectively. What are the dimensions of the fence that will minimize cost?*

Let the lengths of the long and short sides be $l$ and $s$. This is an optimization problem as we want to find a set of $l$ and $s$ that gives us the best cost $c = 2\left(5l+6s\right)$. Try to solve this problem with gradient descent. The solutions are $l=12$ and $s=10$. (Hint: substitute $s = 120l^{-1}$ into the cost first.)

### Before we start

We again use the application of identifying defective metal-casting parts. Let's import the required packages and normalized data:

In [ ]:
from autograd import numpy
from autograd import grad
from matplotlib import pyplot

In [ ]:
"""The following lines imports variables and functions from lesson 5 and 7. The
imported variables and functions include : images_train, images_val, images_test,
labels_train, labels_val, labels_test, neural_network_model, model_loss,
regularized_loss, and performance."""
import sys
sys.path.insert(0, "../scripts")
from load_casting_data import *
from lesson_7_functions import *

In addition, we define a function to return a new set of parameters with Xavier initialization. Every time we train the model with a new optimization method, we want to re-initialize the parameters to the same values to have a fair comparison between different methods. Note that we use a length-one NumPy array for $b^1$ (which is supposed to be a scalar) because then we can use many NumPy functions on it later.

In [ ]:
def get_new_parameters(n0, n1):
    """Get a new set of parameters using a fixed seed and the Xavier initialization.
    
    Arguments
    ---------
    n0, n1 : int
        The number of elements in vector x and the first intermediate vector z1.
    
    Returns
    -------
    A list of 4 elementsb
        W0 : a 2D array of shape (n0, n1).
        b0 : a 1D array of length n1.
        W1 : a 1D array of length n1.
        b1 : a 1D array of only one element (i.e., to mimic a scalar).
    """
    numpy.random.seed(0)  # fix the random seed to avoid "true randomness"
    
    scale = numpy.sqrt(6/(n0+n1))
    W0 = numpy.random.uniform(-scale, scale, (n0, n1))
    b0 = numpy.zeros(n1)

    scale = numpy.sqrt(6/(n1+1))
    W1 = numpy.random.uniform(-scale, scale, n1)
    b1 = numpy.zeros(1)  # using a length-one vector as a scalar
    
    return [W0, b0, W1, b1]

The lengths of each input vector $\mathbf{x}$ and the intermediate vector $\mathbf{z}^1$ are fixed throughout this lesson.

In [ ]:
n0 = images_train.shape[1]  # i.e., total num. of pixels per image
n1 = 64  # the length of z1 vector

Don't forget the function that calculates the gradients for us: 

In [ ]:
gradients = grad(regularized_loss, argnum=2)

## Revisiting gradient descent

In gradient descent, the gradients are the derivatives of a total loss $L$ with respect to parameters. The total loss can be the sum or the mean of the losses from all training samples. For example, we use the mean when doing logistic regression over $N$ training samples:

$$
L\left(\hat{\mathbf{y}}, \mathbf{y}\right)
=
\frac{1}{N} \sum_{i=1}^{N} l\left(\hat{y}_i, y_i\right)
=
\frac{1}{N}\sum_{i=1}^{N} −y_i\log(\hat{y}_i)-(1-y_i)\log(1-\hat{y}_i)
$$

The gradients, in this case, are also the mean of the gradients calculated from the single-sample loss, $l\left(\hat{y}_i, y_i\right)$:

$$
\frac{\mathrm{d}~L}{\mathrm{d}~W}\left(\hat{\mathbf{y}}, \mathbf{y}\right) = \frac{1}{N}\sum_{i=1}^{N}\frac{\mathrm{d}~l}{\mathrm{d}~W}\left(\hat{y}_i, y_i\right)
$$

And the gradient descent algorithm is roughly

```
loop:
    Y_hat 🠘 model(X, W)
    L 🠘 loss(Y_hat, Y)
    W 🠘 W - step_size x (dL/dW)
    exit optimization if model performance is satisfying
```

(Note we never implemented code to calculate `dL/dW` (i.e., $\frac{1}{N}\sum_{i=1}^{N}\frac{\mathrm{d}~l}{\mathrm{d}~W}\left(y_i, \hat{y}_i\right)$) because `autograd` did that for us.)

## Batched and stochastic gradient descent

When we have more samples in the training dataset, an iteration of gradient descent becomes more computationally expensive, i.e., it needs more time and computer memory. This is when ***batched gradient descent*** comes into play.

Batched gradient descent divides an entire training dataset into several batches. Next, it assumes the gradients calculated from the loss of each batch are about the same as the overall gradients:

$$
\frac{\mathrm{d}~L}{\mathrm{d}~W}\left(\hat{\mathbf{y}}, \mathbf{y}\right)
= \frac{1}{N}\sum_{i=1}^{N}\frac{\mathrm{d}~l}{\mathrm{d}~W}\left(\hat{y}_i, y_i\right)
\approx \frac{1}{N_b}\sum_{i=1}^{N_b}\frac{\mathrm{d}~l}{\mathrm{d}~W}\left(\hat{y}_i, y_i\right)
\approx \frac{1}{N_b}\sum_{i=N_b+1}^{2N_b}\frac{\mathrm{d}~l}{\mathrm{d}~W}\left(\hat{y}_i, y_i
\right)
\approx \cdots
$$

$N_b$ denotes the number of samples in each batch. Batches don't need to have the same number of samples. For example, the last batch can be smaller because $N$ may not be divisible by $N_b$.

Some people use batched gradient descent and ***stochastic gradient descent*** interchangeably. Some people, on the other hand, specifically use stochastic gradient descent for the case $N_b=1$. 

The data distribution should be similar in each batch to make batch gradients close enough to overall gradients. For example, let's say we have 800 good casting parts and 200 defective casting parts in our training dataset. In that case, each batch should also have approximately 80%/20% of good/defective casting parts. We can achieve this by ***shuffling*** the training data before dividing them into batches.

In [ ]:
# fix the random seed to avoid "true randomness"
numpy.random.seed(12345)

# create an array for original indices
order = numpy.arange(len(images_train))

# shuffle the indices (`order` is re-ordered in-place)
numpy.random.shuffle(order)

# get re-ordered training data
images_train = images_train[order]
labels_train = labels_train[order]

Note that in the above code cell, the function `numpy.random.shuffle` can shuffle `images_train` directly. However, we first created `new_order` instead because we need to re-order `labels_train` with the same order as `images_train`.

We use $N_b=100$ here. The total number of samples in the training dataset is $782$, so we have $8$ batches (with the last batch only having $82$ samples).

In [ ]:
# define the batch size Nb
Nb = 100

# define the start and end index of each batch
starts = [0, 100, 200, 300, 400, 500, 600, 700]  # python starts from 0
ends = [100, 200, 300, 400, 500, 600, 700, 782]  # python uses end+1 for slicing

Let's examine if each batch has a similar ratio between good and defective casting parts:

In [ ]:
# the overall good-defective ratio in the training dataset
Ndef = numpy.count_nonzero(labels_train)
print("The overall good-defective ratio in images_train: {}".format(
    (len(images_train)-Ndef)/Ndef))

for i, (bg, ed) in enumerate(zip(starts, ends)):
    # calculate and print the ratio
    Ndef = numpy.count_nonzero(labels_train[bg:ed])
    print("Batch {} (samples {} to {}): the ratio is {}".format(
        i, bg, ed-1, (ed-bg-Ndef)/Ndef))

There's no golden rule how close these numbers should be. We assume the good-defective ratios from the batches are close enough to the overall one for the sake of convenience.

When updating the parameters, we loop through batches, calculate the loss of the current batch, get batch gradients, and then update the parameters using the batch gradients:

```
loop:
    loop i = [0, 1, ..., number_of_batches-1]:
        X_b 🠘 training data in the i-th batch
        Y_b 🠘 labels in the i-th batch
        Y_hat_b 🠘 model(X_b, W)
        L_b 🠘 loss(Y_hat_b, Y_b)
        W 🠘 W - step_size x (dL_b/dW)
        exit optimization if model performance is satisfying
```

The `L_b` in the pseudo-code denotes the loss of `i`-th batch, $L_b = \frac{1}{N_b}\sum_{j=iN_b}^{(i+1)N_b}l\left(\hat{y}_j, y_j\right)$, and `dL_b/dW` denotes its gradients.

Let's try the new method to optimize our neural network model:

In [ ]:
%%time

# get a new set of parameters (params=[W0, b0, W1, b1])
params = get_new_parameters(n0, n1)

# step size
lr = 1e-1

# variables to store parameters at when the validation loss is at its lowest
best_val_loss = numpy.inf
best_iter = 0
best_params = None

# variables to store the history of training and validation loss
hist_train_loss = []
hist_val_loss = []

# a counter for the numbero of total iterations
i = 0

# limit the total number of iterations to 2000
while i < 2000:
    
    # loop through each batch
    for j, (bg, ed) in enumerate(zip(starts, ends)):
        
        # alias for data in this batch to shorten the code
        x_b = images_train[bg:ed]
        labels_b = labels_train[bg:ed]
        
        # calculate gradients and use gradient descents
        grads = gradients(x_b, labels_b, params)
        params = [p - lr * g for p, g in zip(params, grads)]
    
        # save losses for future investigation
        hist_train_loss.append(model_loss(x_b, labels_b, params))
        hist_val_loss.append(model_loss(images_val, labels_val, params))
    
        # if the validation loss is lower than the current best, save it
        if hist_val_loss[-1] < best_val_loss:
            best_val_loss = hist_val_loss[-1]
            best_iter = i
            best_params = params
    
        # update iteration counter
        i = i + 1
        
        # print something every 100 iterations so we know it's not dead
        if i % 100 == 0:
            print("{}...".format(i), end="")

# just to add a new line
print()

In [ ]:
print("The best validation loss:", best_val_loss)
print("It happened at iteration {}".format(best_iter))

The optimization took about $100$ seconds to finish $2000$ iterations on a personal desktop with 6 physical cores. Compared to the $2.5$ minutes in lesson 7, it is about $1.5\texttt{x}$ faster. Moreover, we now get the best parameters earlier at the $390$th iteration. If we monitor and stop the optimization in real-time, it means the time-to-solution is $19.5$ seconds. ($100 \times 390 \div 2000 = 19.5$). The time-to-solution of the vanilla gradient descent in lesson 7 is $34.2$ seconds ($2.5 \times 60 \times 456 \div 2000 = 34.2$). So using batched gradient descent improves the time-to-solution with a $1.75\texttt{x}$ speedup!

Just so that you know: it's also possible to further improve the performance by dividing validation data into batches.

We examine the loss history in the following figures:

In [ ]:
plt, ax = pyplot.subplots(1, 2, figsize=(10, 5))
ax[0].plot(range(len(hist_train_loss)), hist_train_loss, label="Training loss")
ax[0].plot(range(len(hist_val_loss)), hist_val_loss, label="Validation loss")
ax[0].set_xlabel("Iteration")
ax[0].set_ylabel("Loss")
ax[0].set_title("Iteration 0 to 1999")
ax[0].legend(loc=0)
ax[1].plot(range(100), hist_train_loss[:100], label="Training loss")
ax[1].plot(range(100), hist_val_loss[:100], label="Validation loss")
ax[1].set_xlabel("Iteration")
ax[1].set_ylabel("Loss")
ax[1].set_title("Iteration 0 to 100")
ax[1].legend(loc=0)
plt.suptitle("Training and validation loss history");

We have the complete loss history on the left. Oscillation happened from the beginning to the end. To have a clearer view of the oscillation, we plotted the first 100 iterations on the right. This oscillation is normal for batched gradient descent. In a nutshell, though we assume the characteristics of a batch (distributions, means, gradients, losses, etc.) are about the same as those of the whole training dataset, they are still different more or less. So using one batch at a time usually gives a bumpier loss history than using the entire dataset.

The smaller the $N_b$ is, the more oscillation we get. The strictly stochastic gradient descent ($N_b=1$) hence usually shows a considerable fluctuation.

The model's final performance:

In [ ]:
# final accuracy
pred_labels_test = classify(images_test, best_params, neural_network_model)
perf = performance(pred_labels_test, labels_test)
print("Final precision: {:.1f}%".format(perf[0]*100))
print("Final recall: {:.1f}%".format(perf[1]*100))
print("Final F-score: {:.1f}%".format(perf[2]*100))

The performance is slightly different from what we got in lesson 7. Many factors contribute to the difference. The most obvious reason is that we were not using the best hyperparameters in both lessons.

Though there are still many other gradient-descent-based optimization methods, we stop the discussion of gradient descent here. Among so many different optimization methods, gradient descent and its variants are the most important ones in modern deep learning. They require fewer computing resources, so optimizing models against millions or even billions of data becomes feasible. We encourage those who want to go deeper with deep learning to check reference [1].

## Using advanced optimizers from SciPy

We have been implementing gradient descent from scratch in all lessons so far. Gradient descent is easy to implement and powerful in many use cases.  However, sometimes we may want to use more advanced methods, but implementing advanced methods may be challenging. For example, [optimizers based on Newton's method](https://en.wikipedia.org/wiki/Quasi-Newton_method) are commonly used for small-size problems but are complicated. It may be more feasible to use third-party packages in such a situation instead of writing the code from scratch.

We want to show you how to use the optimizers from a well-known Python package called SciPy. We will use the [L-BFGS-B optimizer](https://en.wikipedia.org/wiki/Limited-memory_BFGS) to optimize our neural network. L-BFGS-B is based on Newton's method. The primary goal is to show how to use SciPy, and L-BFGS-B serves as an example, so we'll skip the math.

SciPy provides a uniform interface for all optimization methods through [`scipy.optimize.minimize`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.minimize.html). In this lesson, we will use this function with

```
results = scipy.optimize.minimize(
    fun      = <a function returning loss and gradients>,
    x0       = <initial values for parameters>,
    args     = <arguments to `fun`>,
    method   = <optimization method>,
    jac      = <whether `fun` returns gradients,
    callback = <a function to call after every iteration>
)
```

Let's take a look at the arguments:

##### 1. $\texttt{fun}$
   
`fun` is a function that returns both the loss and gradients. This function must have the model parameters as its first argument. All other arguments should be after model parameters. For example, we usually use `params` to represent model parameters in our lessons and use `x` and `y` to represent input data and labels. So if the function name is `loss_and_grad`, then the definition of the function should be something like:
```python
def loss_and_grad(params, x, y):
    ...
    return loss, grads
```

And then `fun=loss_and_grad`.

SciPy expects all parameters to be packed into a flattened 1D array. Previously, our `params` is a `list` of several `numpy.ndarray`. Now we need to flatten each `numpy.ndarray` and concatenate them into a single vector.

##### 2. $\texttt{x0}$

SciPy uses different notation. It uses `x` to denote model parameters while we use `params`. `x0` represents the initial values of model parameters.

##### 3. $\texttt{args}$

All arguments except for the model parameters should be provided through `args`. For example, `loss_and_grad` requires three arguments: model parameters, input data, and labels. During training, input data and labels are `images_train` and `labels_train`, respectively. So `args=(images_train, labels_train)`.

##### 4. $\texttt{method}$

This is a string indicating what optimization method we want to use. [This SciPy tutorial](https://docs.scipy.org/doc/scipy/reference/tutorial/optimize.html#unconstrained-minimization-of-multivariate-scalar-functions-minimize) lists the methods allowed in SciPy. Here we use `mothod='L-BFGS-B'`.
   
##### 5. $\texttt{jac}$

We use `jac=True` to let SciPy know the function provided in `fun` also returns gradients. (`fun` may return the loss only in some situations, and SciPy will then use other approaches to calculate gradients.) Since we already use `autograd` to get gradients, we don't need SciPy to calculate the gradients for us.

##### 6. $\texttt{callback}$

If we provide a function to `callback`, SciPy will call this function after each optimization iteration. The `callback` function takes only one argument: the model parameters at the current optimization iteration. We can then do things like calculating the current validation loss or printing iteration information.

Now let's see how to use SciPy. First, all optimizers are under submodule `scipy.optimize`. We import it first.

In [ ]:
import scipy.optimize

As mentioned, SciPy needs all model parameters to be flattened and packed into a 1D array. We define `join_parameters` to achieve it and `split_parameters` to recover the 1D array back to a list of `numpy.ndarray`s. `split_parameters` takes in three arguments: model parameters, input data sample size, and the length of the intermediate vector. We need the sizes to know the shapes of $W^0$, $\mathbf{b}^0$, $W^1$, and $\mathbf{b}^1$.

In [ ]:
def join_parameters(params):
    """Flatten and concatenate several ndarrays into one single vector.
    
    Arguments
    ---------
    params : a list of numpy.ndarray
        For example, in our neural network, params=[W0, b0, W1, b1].
    
    Returns
    -------
    params : a 1D numpy.ndarray
    """
    params = numpy.concatenate([p.flatten() for p in params])
    return params

In [ ]:
def split_parameters(params, n0, n1):
    """Split an 1D array or parameters into W0, b0, W1, b1.
    
    Arguments
    ---------
    params : 1D numpy.ndarray
        The parameters of a neural network packed in an 1D array.
    n0 : integer
        The number of element in the input vector x.
    n1 : integer
        The number of element in the intermediate vector z1.
    
    Returns
    -------
    A list of 4 elements
        W0 : a 2D array of shape (n0, n1).
        b0 : a 1D array of length n1.
        W1 : a 1D array of length n1.
        b1 : a scalar.
    """
    
    W0 = params[:n0*n1].reshape((n0, n1))
    b0 = params[n0*n1:(n0+1)*n1].reshape((n1,))
    W1 = params[(n0+1)*n1:(n0+2)*n1].reshape((n1,))
    b1 = params[-1]
    return [W0, b0, W1, b1]

Next is what we will provide to `fun`. It is a function taking five arguments: model parameters `params`, input data `x`, labels `y`, size of each input data sample `n0`, and the length of intermediate vector `n1`. The function returns both the loss and gradients. Gradients also need to be flattened and packed into a 1D array.

In [ ]:
def loss_and_grad(params, x, y, n0, n1):
    """A function that returns loss and gradients for using SciPy's optimizers.
    
    Arguments
    ---------
    params : 1D numpy.ndarray
        The parameters of a neural network packed in an 1D array.
    x : numpy.ndarray
        The input data. If a 1D array, it represents the features of one sample. If a
        2D array, the first dimension represents the number of samples, and the second
        dimension represents the features.
    y : numpy.ndarray
        The output of the model. If x is a 1D array, y is a scalar. If x is a 2D array,
        y is a 1D array.
    n0 : integer
        The number of element in the input vector x.
    n1 : integer
        The number of element in the intermediate vector z1.
    
    Returns
    -------
    loss : a scalar
        The total loss (regularized).
    grads : a 1D numpy.ndarray
        The gradients of w.r.t. parameters packed into a single vector.
    """
    # split and recover the shape of parameters
    params = split_parameters(params, n0, n1)
    
    # get the regularized loss (params now is [W0, b0, W1, b1])
    loss = regularized_loss(x, y, params)
    
    # get the gradients (grads is now [dW0, db0, dW1, db1])
    grads = gradients(x, y, params)
    
    # concatenate all gradients into a 1D array
    grads = join_parameters(grads)
    
    return loss, grads

We want to calculate the validation loss at the end of every iteration and record the best model parameters (the ones that give the lowest validation loss). We define the function `best_param_recorder` for this purpose:

In [ ]:
def best_param_recorder(params):
    """A callback function to record the parameters that give the best validation loss.
    
    Arguments
    ---------
    params : 1D numpy.ndarray
        The parameters of a neural network packed in an 1D array.
    
    Returns
    -------
    A boolean. This function always return False to let SciPy optimizers know they can
    continue the optimization.
    
    Notes
    -----
    Variables `best_val_loss` and `best_params` should already exist outside this
    function when this function is called.
    """
    
    # `global` indicates these variables will be the same as those outside the function
    global best_val_loss, best_params
    
    # split and recover the shape of parameters
    params = split_parameters(params, n0, n1)

    # calculate the validation loss
    val_loss = model_loss(images_val, labels_val, params)

    # if the new validation loss is smaller than the current best, save it
    if val_loss <= best_val_loss:
        best_val_loss = val_loss
        best_params = params.copy()

    return False

`best_param_recorder` takes in only model parameters and can only return a boolean. This limitation comes from SciPy. So in order to record the best parameters, we use a keyword `global` in the function. `global` notifies the function that the subsequent variables will be obtained and synchronized with the same variables outside the function, i.e., in the global scope. In this way, we don't have to return the best parameters using `return` but are still able to access them from outside the function. 

In other words, we need to decalre and initialize those global variables first before using the function `best_param_recorder`:

In [ ]:
# we need these variables to be decalred outside the function `best_param_recorder`
best_val_loss = numpy.inf
best_params = None

Finally, we can use `scipy.optimize.minimize` to optimize the model:

In [ ]:
%%time

# get a new set of parameters
params = get_new_parameters(n0, n1)  # params=[W0, b0, W1, b1]
params = join_parameters(params)  # flattened & concatenated

# optimize the model using SciPy
results = scipy.optimize.minimize(
    loss_and_grad, params, args=(images_train, labels_train, n0, n1),
    method='L-BFGS-B', jac=True, tol=1e-9, callback=best_param_recorder)

The returned object `results` contains several information. We can access them through a dot `.`. For example, the number of iterations is `results.nit`:

In [ ]:
print("Number of iterations: {}".format(results.nit))

And the final model parameters are in `results.x` (recall that SciPy uses `x` to denote model parameters). [This webpage](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.OptimizeResult.html#scipy.optimize.OptimizeResult) shows other information carried by `results`. Let's see the performance against the test dataset using the model parameters from the last iteration:

In [ ]:
# final accuracy using the parameters from the last iteration
params = split_parameters(results.x, n0, n1)
pred_labels_test = classify(images_test, params, neural_network_model)
perf = performance(pred_labels_test, labels_test)
print("Final precision: {:.1f}%".format(perf[0]*100))
print("Final recall: {:.1f}%".format(perf[1]*100))
print("Final F-score: {:.1f}%".format(perf[2]*100))

It doesn't look as nice as what we got previously using gradient descent and the batched version. However, remember these numbers are obtained using the model parameters from the last iteration. They are not the parameters that give the lowest validation. Let's calculate the test performance again using the best parameters (i.e., `best_params`):

In [ ]:
# final accuracy using the best parameters
pred_labels_test = classify(images_test, best_params, neural_network_model)
perf = performance(pred_labels_test, labels_test)
print("Final precision: {:.1f}%".format(perf[0]*100))
print("Final recall: {:.1f}%".format(perf[1]*100))
print("Final F-score: {:.1f}%".format(perf[2]*100))

These numbers are different from what we got from gradient descent. It's possible to tune the L-BFGS-B optimizer to get closer numbers. We can find all tunable options for L-BFGS-B optimizer [here](https://docs.scipy.org/doc/scipy/reference/optimize.minimize-lbfgsb.html). We will stop here as our primary goal is to show you how to use SciPy's optimizers.

## Challenge

Recall the 120-square-feet fence we wanted to build at the beginning of this notebook. Can you write code to solve the same problem using SciPy's optimizers?

## What we've learned

1. Using batched gradient descent to accelerate the optimization.
2. Using third-party libaraies, like SciPy, to optimize a model with advanced optimizers.

## Reference
1. Goodfellow, I., Bengio, Y., & Courville, A. (2016). Chapter 8. Optimization for Training Deep Models. In Deep Learning (pp. 271–325). MIT Press. http://www.deeplearningbook.org

In [ ]:
# Execute this cell to load the notebook's style sheet, then ignore it
from IPython.core.display import HTML
css_file = '../style/custom.css'
HTML(open(css_file, "r").read())